### Neural Network - Two Hidden Layers

In [1]:

# Import our dependencies
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
import pandas as pd
import tensorflow as tf
from pathlib import Path

In [2]:
# Disable warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
def preprocessFile(filename = Path('../Data/AdultCensusUpdated.csv')):
    
    print(f'Preprocessing file {filename}')
    dtype_mapper = {'age': 'int64',
                    'workclass': 'string',
                    'fnlwgt': 'int64',
                    'education': 'string',
                    'education.num': 'int64',
                    'marital.status': 'string',
                    'occupation': 'string',
                    'relationship': 'string',
                    'race': 'string',
                    'sex': 'string',
                    'capital.gain': 'int64',
                    'capital.loss': 'int64',
                    'hours.per.week': 'int64',
                    'native.country': 'string',
                    'income': 'string',
                    'State': 'string'}
    df = pd.read_csv(filename, dtype=dtype_mapper, na_values='?')    
    df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'race', 'sex',
       'capital_gain', 'capital_loss', 'hours_per_week', 'native_country',
       'income', 'state']
    df.income = df.income.apply(lambda x: '0' if x == '<=50K' else '1')
    df.income = df.income.astype('int64')
    df = df.drop(['fnlwgt',
                  'education', 
                #   'state',
                  'capital_gain', 
                  'capital_loss',
                  'native_country'], axis=1)
    df = df.dropna()
    return df

census_df = preprocessFile()
census_df.sample(10)

Preprocessing file ..\Data\AdultCensusUpdated.csv


age         workclass  education_num      marital_status  \
9566    30           Private             10  Married-civ-spouse   
2529    45         Local-gov             10  Married-civ-spouse   
7831    31  Self-emp-not-inc              5  Married-civ-spouse   
1191    27           Private             10  Married-civ-spouse   
31208   41           Private              9           Separated   
28215   50           Private              7            Divorced   
20777   25           Private             13  Married-civ-spouse   
19872   19           Private             10       Never-married   
27932   26           Private             10       Never-married   
2673    36           Private             13  Married-civ-spouse   

              occupation   relationship   race     sex  hours_per_week  \
9566        Craft-repair        Husband  White    Male              40   
2529      Prof-specialty        Husband  White    Male              50   
7831        Craft-repair        Husband  White    Male              60   
1191   Machine-op-inspct        Husband  White    Male              40   
31208  Handlers-cleaners  Not-in-family  Black    Male              40   
28215      Other-service      Unmarried  White  Female              40   
20777     Prof-specialty        Husband  White    Male              45   
19872      Other-service      Own-child  White  Female              18   
27932              Sales  Not-in-family  Black  Female              37   
2673       Other-service        Husband  Black    Male              36   

       income          state  
9566        0       Illinois  
2529        1      Louisiana  
7831        0        Montana  
1191        0  Massachusetts  
31208       0      Tennessee  
28215       0           Ohio  
20777       1       New York  
19872       0       Maryland  
27932       0         Hawaii  
2673        1        Georgia

In [4]:
y = census_df.income

X = census_df.copy()
X.drop('income', axis=1, inplace=True)


In [5]:
# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Apply Label Encoding to each non-numeric column
for col in X.select_dtypes(include=["string"]).columns:
    X[col] = label_encoder.fit_transform(X[col])

X.head()

age  workclass  education_num  marital_status  occupation  relationship  \
1   82          2              9               6           3             1   
3   54          2              4               0           6             4   
4   41          2             10               5           9             3   
5   34          2              9               0           7             4   
6   38          2              6               5           0             4   

   race  sex  hours_per_week  state  
1     4    0              18     19  
3     4    0              40     11  
4     4    0              40      8  
5     4    0              45      8  
6     4    1              40      4

In [6]:
# Use sklearn to split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Create scaler instance
X_scaler = scaler.fit(X_train)

# Fit the scaler
X_scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [7]:
X_train

age  workclass  education_num  marital_status  occupation  \
20085   56          2              9               2           6   
24330   45          2              9               0           0   
5560    26          5              9               0           0   
16498   28          2             10               4          13   
14880   51          2              9               2          10   
...    ...        ...            ...             ...         ...   
18299   62          0              9               6           0   
5454    48          2              9               6           7   
12883   48          2             13               0           7   
252     26          2             10               2           0   
31499   58          2             14               6          11   

       relationship  race  sex  hours_per_week  state  
20085             0     4    1              40     16  
24330             4     4    0              28     24  
5560              3     4    0              30     36  
16498             1     4    1              50     31  
14880             0     4    1              40     36  
...             ...   ...  ...             ...    ...  
18299             4     4    0              40     31  
5454              4     4    0              40      8  
12883             4     4    0              56     49  
252               0     4    1              40     24  
31499             1     4    0              27     15  

[23038 rows x 10 columns]

In [8]:
# Define the deep learning model 
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  64
hidden_nodes_layer2 = 32

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                704       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2817 (11.00 KB)
Trainable params: 2817 (11.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Compile the model
# Compile the Sequential model together and customize metrics
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [10]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=10, batch_size=32)


Epoch 1/10
720/720 [==============================] - 1s 807us/step - loss: 0.3938 - accuracy: 0.8115
Epoch 2/10
720/720 [==============================] - 1s 782us/step - loss: 0.3651 - accuracy: 0.8267
Epoch 3/10
720/720 [==============================] - 1s 850us/step - loss: 0.3615 - accuracy: 0.8285
Epoch 4/10
720/720 [==============================] - 1s 789us/step - loss: 0.3593 - accuracy: 0.8313
Epoch 5/10
720/720 [==============================] - 1s 781us/step - loss: 0.3577 - accuracy: 0.8310
Epoch 6/10
720/720 [==============================] - 1s 765us/step - loss: 0.3561 - accuracy: 0.8302
Epoch 7/10
720/720 [==============================] - 1s 851us/step - loss: 0.3545 - accuracy: 0.8323
Epoch 8/10
720/720 [==============================] - 1s 780us/step - loss: 0.3537 - accuracy: 0.8328
Epoch 9/10
720/720 [==============================] - 1s 849us/step - loss: 0.3523 - accuracy: 0.8327
Epoch 10/10
720/720 [==============================] - 1s 815us/step - loss: 0.351

In [11]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

240/240 - 0s - loss: 0.3744 - accuracy: 0.8236 - 275ms/epoch - 1ms/step
Loss: 0.3744143843650818, Accuracy: 0.8235676884651184


In [12]:
y_predictions = nn.predict(X_test_scaled)
y_predictions = tf.squeeze(y_predictions)
y_predictions_binary = np.array([1 if x >= 0.5 else 0 for x in y_predictions])
actual = np.array(y_test)
cm = confusion_matrix(actual, y_predictions_binary)
displ = ConfusionMatrixDisplay(confusion_matrix=cm)
displ.plot()


240/240 [==============================] - 0s 573us/step


In [13]:
y_predictions_binary

array([1, 1, 1, ..., 0, 0, 0])

In [14]:
# Calculating the confusion matrix
cm = confusion_matrix(y_test, y_predictions_binary, normalize='pred')
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, y_predictions_binary)

In [15]:
# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, y_predictions_binary))

Confusion Matrix


Predicted 0  Predicted 1
Actual 0     0.872438     0.338773
Actual 1     0.127562     0.661227

Accuracy Score : 0.8235677083333334
Classification Report
              precision    recall  f1-score   support

           0       0.87      0.90      0.88      5752
           1       0.66      0.61      0.63      1928

    accuracy                           0.82      7680
   macro avg       0.77      0.75      0.76      7680
weighted avg       0.82      0.82      0.82      7680



### Export Classification Report

In [17]:
import csv

def classificationReportToPanda(report, model):
    file = f'../Data/classification_report_{model}.csv'
    df = pd.DataFrame(report).transpose()
    df['model'] = model
    df['model'] = df['model'].astype('string')
    df['support'] = df['support'].astype('int64')    
    df.to_csv(file, index_label='class', quoting=csv.QUOTE_ALL)
    display(df)

report = classification_report(y_test, y_predictions_binary, output_dict=True)

classificationReportToPanda(report, 'NeuralNetwork')

precision    recall  f1-score  support          model
0              0.872438  0.895341  0.883741     5752  NeuralNetwork
1              0.661227  0.609440  0.634278     1928  NeuralNetwork
accuracy       0.823568  0.823568  0.823568        0  NeuralNetwork
macro avg      0.766832  0.752390  0.759009     7680  NeuralNetwork
weighted avg   0.819415  0.823568  0.821115     7680  NeuralNetwork

### Add Test Results to Original Dataframe

In [18]:
X_test['Results'] = y_predictions_binary
X_test['Model'] = 'Sequential NN'

# Drop unnecessary columns
X_test = X_test.drop(columns = ['age', 'workclass', 'education_num', 'marital_status', 'occupation', 
            'relationship', 'race', 'sex', 'hours_per_week', 'state'])

X_test.head()

Results          Model
3314         1  Sequential NN
19677        1  Sequential NN
29448        1  Sequential NN
3581         0  Sequential NN
25490        0  Sequential NN

In [19]:
# Copy original reprocess dataset
census_copy_df = census_df.copy()

#Join result data to original
census_copy_df = census_copy_df.join(X_test)
census_copy_df = census_copy_df.dropna(axis=0, how='any')
census_copy_df.head(20)

age         workclass  education_num         marital_status  \
1    82           Private              9                Widowed   
4    41           Private             10              Separated   
13   32           Private             14              Separated   
16   45           Private              7               Divorced   
22   61           Private              9               Divorced   
26   33           Private              2     Married-civ-spouse   
27   49           Private              3     Married-civ-spouse   
28   37      Self-emp-inc              6          Never-married   
30   23           Private             10              Separated   
36   53           Private              9          Never-married   
37   51           Private             15          Never-married   
38   37  Self-emp-not-inc             16          Never-married   
40   44           Private             13               Divorced   
41   43           Private             14               Divorced   
42   51           Private             10               Divorced   
45   48           Private             13  Married-spouse-absent   
51   50           Private              9                Widowed   
52   51         State-gov             11               Divorced   
53   42           Private             10          Never-married   
63   51  Self-emp-not-inc             13     Married-civ-spouse   

          occupation   relationship                race     sex  \
1    Exec-managerial  Not-in-family               White  Female   
4     Prof-specialty      Own-child               White  Female   
13   Exec-managerial  Not-in-family               White    Male   
16  Transport-moving  Not-in-family               White    Male   
22             Sales      Unmarried               White  Female   
26      Craft-repair  Not-in-family               White    Male   
27     Other-service        Husband               White    Male   
28  Transport-moving  Not-in-family               White    Male   
30     Other-service  Not-in-family               White    Male   
36             Sales  Not-in-family               White    Male   
37    Prof-specialty  Not-in-family               White  Female   
38    Prof-specialty  Not-in-family               White  Female   
40   Exec-managerial      Unmarried               White    Male   
41   Exec-managerial      Unmarried               White  Female   
42      Adm-clerical      Unmarried               White  Female   
45             Sales      Unmarried               White  Female   
51             Sales  Not-in-family               White  Female   
52      Tech-support  Not-in-family               White    Male   
53   Exec-managerial  Not-in-family               White  Female   
63             Sales        Husband  Asian-Pac-Islander    Male   

    hours_per_week  income           state  Results          Model  
1               18       0        Maryland      0.0  Sequential NN  
4               40       0         Florida      0.0  Sequential NN  
13              55       1   New Hampshire      0.0  Sequential NN  
16              76       1         Wyoming      0.0  Sequential NN  
22              25       0        New York      0.0  Sequential NN  
26              32       0        Colorado      0.0  Sequential NN  
27              40       0         Montana      0.0  Sequential NN  
28              50       1        Colorado      0.0  Sequential NN  
30              40       1        New York      0.0  Sequential NN  
36              48       1            Utah      0.0  Sequential NN  
37              50       1         Arizona      0.0  Sequential NN  
38              60       1       Tennessee      0.0  Sequential NN  
40              50       1            Utah      1.0  Sequential NN  
41              40       1         Indiana      0.0  Sequential NN  
42              40       1          Kansas      0.0  Sequential NN  
45              70       1        Maryland      0.0  Sequential NN  
51             

In [20]:
# Export
census_copy_df.to_csv('../Data/SequentialNN.csv', index_label='index')
